<a href="https://colab.research.google.com/github/8percent/programming-bitcoin/blob/main/chapter10/%EC%98%A4%EC%9E%AC%EA%B7%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/jimmysong/programmingbitcoin

Cloning into 'programmingbitcoin'...
remote: Enumerating objects: 6406, done.
remote: Total 6406 (delta 0), reused 0 (delta 0), pack-reused 6406
Receiving objects: 100% (6406/6406), 39.43 MiB | 25.14 MiB/s, done.
Resolving deltas: 100% (4254/4254), done.


In [2]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
cd programmingbitcoin/code-ch10

/content/programmingbitcoin/code-ch10


In [5]:
ls

answers.py  Chapter10.ipynb  examples.py  jupyter.txt  op.py         script.py
block.py    ecc.py           helper.py    network.py   __pycache__/  tx.py


In [6]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import network

from block import GENESIS_BLOCK
from helper import calculate_new_bits
from network import (
    NetworkEnvelope,
    VersionMessage,
)

### Exercise 1

Determine what this network message is:

`f9beb4d976657261636b000000000000000000005df6e0e2`

In [20]:
# Exercise 1
reload(network)
from network import NetworkEnvelope
from io import BytesIO
message_hex = 'f9beb4d976657261636b000000000000000000005df6e0e2'
stream = BytesIO(bytes.fromhex(message_hex))
env = NetworkEnvelope.parse(stream)
print(env.command)
print(env.payload)
# convert to binary
# see what the bytes 4 through 16 aregi

b'verack'
b''


### Exercise 2

Write the `parse` method for `NetworkEnvelope`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:NetworkEnvelopeTest:test_parse`

In [11]:
# Exercise 2

reload(network)
run(network.NetworkEnvelopeTest("test_parse"))

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


### Exercise 3

Write the `serialize` method for `NetworkEnvelope`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:NetworkEnvelopeTest:test_serialize`

In [12]:
# Exercise 3

reload(network)
run(network.NetworkEnvelopeTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Exercise 4

Write the `serialize` method for `VersionMessage`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:VersionMessageTest:test_serialize`

In [22]:
# Exercise 4

reload(network)
run(network.VersionMessageTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### Exercise 5

Write the `handshake` method for `SimpleNode`

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:SimpleNodeTest:test_handshake`

In [27]:
# Exercise 5

reload(network)
run(network.SimpleNodeTest("test_handshake"))

.
----------------------------------------------------------------------
Ran 1 test in 0.114s

OK


### Exercise 6

Write the `serialize` method for `GetHeadersMessage`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:GetHeadersMessageTest:test_serialize`

In [25]:
# Exercise 6

reload(network)
run(network.GetHeadersMessageTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [28]:
from io import BytesIO
from network import SimpleNode, GetHeadersMessage, HeadersMessage
from block import Block, GENESIS_BLOCK, LOWEST_BITS
from helper import calculate_new_bits
previous = Block.parse(BytesIO(GENESIS_BLOCK))
first_epoch_timestamp = previous.timestamp
expected_bits = LOWEST_BITS
count = 1
node = SimpleNode('mainnet.programmingbitcoin.com', testnet=False)
node.handshake()
for _ in range(19):
    getheaders = GetHeadersMessage(start_block=previous.hash())
    node.send(getheaders)
    headers = node.wait_for(HeadersMessage)
    for header in headers.blocks:
        if not header.check_pow():
            raise RuntimeError('bad PoW at block {}'.format(count))
        if header.prev_block != previous.hash():
            raise RuntimeError('discontinuous block at {}'.format(count))
        if count % 2016 == 0:
            time_diff = previous.timestamp - first_epoch_timestamp
            expected_bits = calculate_new_bits(previous.bits, time_diff)
            print(expected_bits.hex())
            first_epoch_timestamp = header.timestamp
        if header.bits != expected_bits:
            raise RuntimeError('bad bits at block {}'.format(count))
        previous = header
        count += 1

ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
ffff001d
6ad8001d
28c4001d
71be001d
